***1- Data Management.*** <br>
https://dvc.org/doc/start
***

1- Initializee the dvc repository. <br>
2- commit what we did so far. <br>
3- download the data with dvc get wrapper. <br>
4- add data to dvc tracking. this line will create a file with same name as our data with .dvc format and also a .gitignore file. <br>
5- add the tracking data and git placeholder with git command. <br>
6- create the remote locally or online. add the remote as a dvc remote. <br>
7- push the data using dvc order and follow by a git commit to save the version control. <br>
8- change between different version of the model and data. <br>
9- just use these codes to add and commit new modified datasets. <br>

In [2]:
! dvc init


Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


In [4]:
! git status

On branch master
Your branch is up to date with 'origin/master'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	new file:   .dvc/.gitignore
	new file:   .dvc/config
	new file:   .dvcignore

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.ipynb_checkpoints/
	MLOPs.ipynb



In [5]:
! git commit -m "Initialize DVC"
# download the data and save it in a new folder
! dvc get https://github.com/iterative/dataset-registry \
          get-started/data.xml -o data/data.xml


[master d9730c7] Initialize DVC
 3 files changed, 6 insertions(+)
 create mode 100644 .dvc/.gitignore
 create mode 100644 .dvc/config
 create mode 100644 .dvcignore
                                                                            

In [6]:
! dvc add data/data.xml

⠋ Checking graph                                       core>
Adding...                                                                       
!
                                                                                
!
  0% Checking cache in '/home/ali/NPEMF/.dvc/cache/files/md5'| |0/? [00:00<?,   
                                                                                
!
  0%|          |Adding data/data.xml to cache         0/1 [00:00<?,     ?file/s]
                                                                                
!
  0%|          |Checking out /home/ali/NPEMF/data/data0/1 [00:00<?,    ?files/s]
100% Adding...|████████████████████████████████████████|1/1 [00:00, 13.18file/s]

To track the changes with git, run:

	git add data/.gitignore data/data.xml.dvc

To enable auto staging, run:

	dvc config core.autostage true


In [7]:
! git add data/data.xml.dvc data/.gitignore
! git commit -m "Add raw data"

[master 3560319] Add raw data
 2 files changed, 6 insertions(+)
 create mode 100644 data/.gitignore
 create mode 100644 data/data.xml.dvc


In [8]:
# local storage
! mkdir /tmp/dvcstore
! dvc remote add -d myremote /tmp/dvcstore

# remote storage
# ! dvc remote add -d storage s3://mybucket/dvcstore

Setting 'myremote' as a default remote.


In [10]:
! dvc push
# ! dvc pull
! git commit data/data.xml.dvc -m "Dataset updates"
! git push

Pushing
!
  0% Checking cache in '/tmp/dvcstore/files/md5'|    |0/? [00:00<?,    ?files/s]
Pushing                                                                         
Everything is up to date.
On branch master
Your branch is ahead of 'origin/master' by 2 commits.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   .dvc/config

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.ipynb_checkpoints/
	MLOPs.ipynb

no changes added to commit (use "git add" and/or "git commit -a")
Enter passphrase for key '/home/ali/.ssh/id_ed25519': 


In [ ]:
# change to an specific version
! git checkout 72de82ee151cae08636f90435be6709a8632ac96 # HEAD~1 data/data.xml.dvc # this line will checkout one of the git commits.
! dvc checkout # and then dvc checkout makes sure that the data is correctly checked out.

# After checking the data, to checkout to the last version of the data. use the following codes.
! git checkout main  # or whatever branch you were on
! dvc checkout  # to restore the latest version of the data

In [ ]:
# Use the following after data modification.
! dvc add data.xml
! git add data.xml.dvc .gitignore
! git commit -m 'test the data change'
! dvc push
! git push

***2- Data Quality Assurance:***
***
**2-1- Handling Missing Data and Time Zone Alignment:**<br>
Imputation: We use mean, median, or forward filling based on historical data for numerical fields like SpotPriceDKK or SpotPurchase. <br>
Data Exclusion: If certain columns are irrelevant for specific models, they can be excluded from training.
Time Zone Alignment: We standardize all timestamps to DK using Pandas, ensuring consistent data input for the model.

In [ ]:
# Importing the data from excel and showing the info about whole data
import pandas as pd

data_path= './data/NordPoolMarket.xlsx'
raw_data= pd.read_excel(data_path)
missing_values= raw_data.isnull()
missing_counts= missing_values.sum() 
row_with_missing_values= raw_data[raw_data.isnull().any(axis=1)]
print('A summary about dataset')
print(raw_data.info())
print('Missing values in the dataset')
print(missing_values)
print('The number of missing values in the dataset')
print(missing_counts)
print('Rows with the missing values')
print(row_with_missing_values)

In [ ]:
raw_data_1= raw_data.drop(columns='HourUTC') # removing UTC time to proceed just with DK time.
# Fill in the missing values using interpolation for elbas datas and mean for others.
raw_data_1['ElbasAveragePriceDKK']= raw_data_1['ElbasAveragePriceDKK'].interpolate(method='linear') 
raw_data_1['ElbasMaxPriceDKK']= raw_data_1['ElbasMaxPriceDKK'].interpolate(method='linear') 
raw_data_1['ElbasMinPriceDKK']= raw_data_1['ElbasMinPriceDKK'].interpolate(method='linear')
raw_data_1['ElbasAveragePriceEUR']= raw_data_1['ElbasAveragePriceEUR'].interpolate(method='linear') 
raw_data_1['ElbasMaxPriceEUR']= raw_data_1['ElbasMaxPriceEUR'].interpolate(method='linear') 
raw_data_1['ElbasMinPriceEUR']= raw_data_1['ElbasMinPriceEUR'].interpolate(method='linear')
print(raw_data_1.isnull().sum())
# We can see that there are still nul cells in the dataframe
raw_data_1= raw_data_1.drop([0,1]) # removing the first 2 rows because they don't have value
print(raw_data_1.isnull().sum())

# Save the modified excel file with modified added to the end.
revised_path= data_path[:-4]+'modified'+'.xlsx'
raw_data_1.to_excel(revised_path, index=False)

**2-1- Great Expectation Check:**
***
With Great Expectations, we can define different metrics to be assessed automatically in the pipeline. The pipeline includes the validation of data formats, ensuring the data integrity, and checking the compliance with schema requirements before training the model.



In [9]:

import datetime

import pandas as pd

import great_expectations as gx
import great_expectations.jupyter_ux
from great_expectations.core.expectation_configuration import ExpectationConfiguration
from great_expectations.data_context.types.resource_identifiers import ExpectationSuiteIdentifier
from great_expectations.exceptions import DataContextError

context = gx.get_context()
# context= context.convert_to_file_context()
validator= context.sources.pandas_default.read_excel(data_path)

expectation_suite_name = "Energy Data Analysis"
try:
    suite = context.get_expectation_suite(expectation_suite_name=expectation_suite_name)
    print(f'Loaded ExpectationSuite "{suite.expectation_suite_name}" containing {len(suite.expectations)} expectations.')
except DataContextError:
    suite = context.add_expectation_suite(expectation_suite_name=expectation_suite_name)
    print(f'Created ExpectationSuite "{suite.expectation_suite_name}".')

validator.expect_column_values_to_be_between(column='ElbasMinPriceEUR', min_value= 1,
                                            max_value=10)
validator.save_expectation_suite(discard_failed_expectations=False)
checkpoint = context.add_or_update_checkpoint(
    name=expectation_suite_name,
    validator=validator)
checkpoint_result = checkpoint.run()
context.view_validation_result(checkpoint_result)
# context.build_data_docs()
# docs_path = context.get_docs_sites_urls()[0]['site_url']
# print(f"Open this URL in your browser to view data docs: {docs_path}")
# context.open_data_docs()

print(context.get_expectation_suite(expectation_suite_name=expectation_suite_name))
context.add_or_update_expectation_suite(expectation_suite=suite)

suite_identifier = ExpectationSuiteIdentifier(expectation_suite_name=expectation_suite_name)
context.build_data_docs(resource_identifiers=[suite_identifier])
docs_path = context.get_docs_sites_urls()[0]['site_url']
print(f"Open this URL in your browser to view data docs: {docs_path[:7]+'wsl.localhost/Debian'+docs_path[7:]}")
context.open_data_docs(resource_identifier=suite_identifier)

2024-10-25T14:25:42+0200 - INFO - FileDataContext loading fluent config
2024-10-25T14:25:42+0200 - INFO - Loading 'datasources' ->
[{'name': 'default_pandas_datasource', 'type': 'pandas'}]
2024-10-25T14:25:42+0200 - INFO - Of 1 entries, no 'datasources' could be loaded
2024-10-25T14:25:42+0200 - INFO - Saving 1 Fluent Datasources to /home/ali/NPEMF/gx/great_expectations.yml
2024-10-25T14:25:42+0200 - INFO - PandasDatasource.dict() - missing `config_provider`, skipping config substitution
2024-10-25T14:25:42+0200 - INFO - Saving 1 Fluent Datasources to /home/ali/NPEMF/gx/great_expectations.yml
2024-10-25T14:25:42+0200 - INFO - ExcelAsset.dict() - missing `config_provider`, skipping config substitution
2024-10-25T14:25:42+0200 - INFO - PandasDatasource.dict() - missing `config_provider`, skipping config substitution
Loaded ExpectationSuite "Energy Data Analysis" containing 0 expectations.


Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

2024-10-25T14:25:51+0200 - INFO - 	1 expectation(s) included in expectation_suite.
2024-10-25T14:25:59+0200 - INFO - 	1 expectation(s) included in expectation_suite.


Calculating Metrics:   0%|          | 0/10 [00:00<?, ?it/s]

{
  "expectation_suite_name": "Energy Data Analysis",
  "ge_cloud_id": null,
  "expectations": [],
  "data_asset_type": null,
  "meta": {
    "great_expectations_version": "0.18.19"
  }
}
Open this URL in your browser to view data docs: file://wsl.localhost/Debian/home/ali/NPEMF/gx/uncommitted/data_docs/local_site/index.html
